In [4]:
import os
from cv2 import cv2
import numpy as np
import tensorflow.keras.applications as applications
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Lambda
from tensorflow.keras import backend as K

In [5]:

def euclidean_distance(x, y):
    sum_square = K.sum(K.square(x - y), axis=0, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))


def cosine_distance(x, y):
    x = K.l2_normalize(x, axis=-1)
    y = K.l2_normalize(y, axis=-1)
    return -K.mean(x * y, axis=-1, keepdims=True)


def l1_distance(x, y):
    return K.sum(K.abs(x - y), axis=1, keepdims=True)


def dist_mode(x, y, mode):
    if mode == 'euclidean':
        return euclidean_distance(x, y)
    elif mode == 'cosine':
        return cosine_distance(x, y)
    elif mode == 'l1':
        return l1_distance(x, y)

def triplet_loss(mode, margin=0.6):
    def loss(y_true, y_pred):
        anchor = y_pred[0]
        positive = y_pred[1]
        negative = y_pred[2]
        pos_dist = dist_mode(positive, anchor, mode)
        neg_dist = dist_mode(negative, anchor, mode)
        basic_loss = pos_dist - neg_dist + margin
        return K.maximum(basic_loss, 0.0)
    return loss


In [6]:
input_shape = [218, 178, 3]
batch_size = 1
mode = 'euclidean'
mobile_net = applications.MobileNetV2(weights=None, include_top=False, input_shape=input_shape)

anchor = Input(shape=input_shape)
positive = Input(shape=input_shape)
negative = Input(shape=input_shape)

processed_a = mobile_net(anchor)
processed_p = mobile_net(positive)
processed_n = mobile_net(negative)

model = Model(inputs=[anchor, positive, negative], outputs=[processed_a, processed_p, processed_n])

In [7]:
dummy_input = np.zeros(input_shape)
dummy_input = dummy_input[None, ...].astype(np.float32)
model([dummy_input, dummy_input, dummy_input])


[<tf.Tensor: id=18187, shape=(1, 7, 6, 1280), dtype=float32, numpy=
 array([[[[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],
 
         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],
 
         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],
 
         ...,
 
         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
     

In [31]:
t = input_shape
t.insert(0, batch_size)
t = tuple(t)
anc = np.ndarray(shape=t)
pos = np.ndarray(shape=t)
neg = np.ndarray(shape=t)
ind = 0

t1 = [1000]
# t1.insert(0, 3)
t1.insert(0, batch_size)
dummy = np.ndarray(shape=tuple(t1))
dummy.shape

(1, 1000)

(1, 3, 1000)

In [25]:
model([dummy_input, dummy_input, dummy_input])

[<tf.Tensor 'model_1/mobilenetv2_1.00_218_2/Logits/Softmax:0' shape=(1, 1000) dtype=float32>,
 <tf.Tensor 'model_1/mobilenetv2_1.00_218_1/Logits/Softmax:0' shape=(1, 1000) dtype=float32>,
 <tf.Tensor 'model_1/mobilenetv2_1.00_218/Logits/Softmax:0' shape=(1, 1000) dtype=float32>]

In [19]:
model.compile(loss=triplet_loss(mode=mode), optimizer='adam')

In [26]:
model.train_on_batch([dummy_input, dummy_input, dummy_input], [dummy_output, dummy_output, dummy_output])

InvalidArgumentError: 2 root error(s) found.
  (0) Invalid argument: slice index 1 of dimension 0 out of bounds.
	 [[{{node loss_1/mobilenetv2_1.00_218_loss/strided_slice_1}}]]
	 [[loss_1/Identity_2/_2443]]
  (1) Invalid argument: slice index 1 of dimension 0 out of bounds.
	 [[{{node loss_1/mobilenetv2_1.00_218_loss/strided_slice_1}}]]
0 successful operations.
0 derived errors ignored.

In [32]:
dummy_output.shape

(1, 1000)

In [33]:
dummy_input.shape

(1, 218, 178, 3)

In [1]:
def generator(input_shape, output_shape, batch_size=64, word='train'):
    t = input_shape
    t.insert(0, 3)
    t.insert(0, batch_size)
    train_batch = np.ndarray(shape=tuple(t))
    ind = 0

    t1 = output_shape
    t1.insert(0, 3)
    t1.insert(0, batch_size)
    dummy = np.ndarray(shape=tuple(t1))

    ind = 0
    a = random_indice(f"/home/tung/final-thesis/data/{word}")
    for positives in a:
        a_positive = random_indice(f"/home/tung/final-thesis/data/{word}/{positives}")
        for anchor in a_positive:
            img_a = cv2.imread(f"/home/tung/final-thesis/data/{word}/{positives}/{anchor}")
            for positive in a_positive:
                if (anchor != positive):
                    img_p = cv2.imread(f"/home/tung/final-thesis/data/{word}/{positives}/{positive}")
                    for negatives in a:
                        if (negatives != positives):
                            for negative in negatives:
                                img_n = cv2.imread(f"/home/tung/final-thesis/data/{word}/{negatives}/{negative}")
                                train_batch[ind] = [img_a, img_p, img_n]
                                ind = (ind + 1) % batch_size
                                if (ind == 0):
                                    yield train_batch, dummy